In [1]:
import tensorflow as tf
import keras

/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/tensorf

### MNIST DATA

In [2]:
# Import the dataset
from keras.datasets import mnist

# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

# Import the backend
from keras import backend as K

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 18s 2us/step


In [4]:
x_train

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

### Multi Layer Perceptron
Let's start with a kind of neural network we've seen before: a multi-layer perceptron. Recall from our previous neural networks sections that this is a set of perceptron models organized into layers, one layer feeding into the next.

To do this, we will first need to reshape our data into flat vectors for each digit. We'll also need to convert our outcome to a matrix of binary variables, rather than the digit.

In [5]:
# Change shape 
# Note that our images are 28*28 pixels, so in reshaping to arrays we want
# 60,000 arrays of length 784, one for each image
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# Convert to float32 for type consistency
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values to 1 from 0 to 255 (256 values of pixels)
x_train /= 255
x_test /= 255

# Print sample sizes
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
# So instead of one column with 10 values, create 10 binary columns
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

60000 train samples
10000 test samples


Great. Now we can create our model. We'll do this using dense layers and dropouts. Dense layers are simply fully connected layers with a given number of perceptrons. Dropout drops a certain portion of our perceptrons in order to prevent overfitting. Our activation function, `relu` stands for Rectified Linear Unit, which is standard but can be read about more [here](https://en.wikipedia.org/wiki/Rectifier_(neural_networks).

In [6]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


Now we have a model. This we can use to accomplish our wildest dreams of data modeling, or at least predict some digits from pixel data. To do that we will use epochs, effectively iterations of the model, improving based on what it learned previously. Batch size is the number of samples to use in each step improving the model and will affect speed, but also slightly negatively impact accuracy (learning in bigger steps will affect what your model learns).

Note that we are going with 64 perceptron wide layers, this is relatively arbitrary, though units within the $2^x$ series will parallelize more efficiently. Also note that our number of parameters is the product of our input width plus one and our layer width. This reflects the number of weights we're creating in that layer.

In [7]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 54us/step - loss: 0.4330 - acc: 0.8748 - val_loss: 0.1946 - val_acc: 0.9428
Epoch 2/10
60000/60000 [==============================] - 3s 45us/step - loss: 0.2061 - acc: 0.9386 - val_loss: 0.1393 - val_acc: 0.9580
Epoch 3/10
60000/60000 [==============================] - 2s 41us/step - loss: 0.1588 - acc: 0.9532 - val_loss: 0.1156 - val_acc: 0.9637
Epoch 4/10
60000/60000 [==============================] - 2s 39us/step - loss: 0.1307 - acc: 0.9612 - val_loss: 0.1041 - val_acc: 0.9686
Epoch 5/10
60000/60000 [==============================] - 3s 47us/step - loss: 0.1129 - acc: 0.9656 - val_loss: 0.0940 - val_acc: 0.9723
Epoch 6/10
60000/60000 [==============================] - 3s 42us/step - loss: 0.1033 - acc: 0.9689 - val_loss: 0.0886 - val_acc: 0.9739
Epoch 7/10
60000/60000 [==============================] - 2s 38us/step - loss: 0.0935 - acc: 0.9709 - val_loss: 0.0902 - val_acc

That did impressively well for such a simple neural network, with each epoch training in about 1 second on this machine and giving us an accuracy in the high 90's. But what else can we do? Let's let our model get much more complicated by introducting convolution.

## Convolutional Neural Networks

Before we go any further, do you recall that we've talked about how complex neural networks can get, and the degree of computational complexity that entails? Well, here we're going to finally truly experience that complexity, so be careful about rerunning this code. It will take some serious time (potentially on the order of hours) to run.

Now that that's out of the way, let's talk convolution. First, a simple definition. Convolution basically takes your data and creates overlapping subsegments testing for a given feature in a set of spaces and upon which it develops its model.

Let's extend that definition since it's incredibly dense.

First, you have to define a shape of your input data. This can theoretically be in any number of dimensions, though for our image example we will use 2d, since images are in two dimensions. This is also why you'll see 2D in some of our layer definitions (though more on that later). Our first chunk of code after loading the data does this reshaping (with a conditional on the data format).

Over that shaped data, we then create our tiles, also called __kernels__. These kernels are like little windows, that will look over subsets of the data of a given size. In the example below we create 3x3 kernels, which run overlapping over the whole 28x28 input looking for features. That is the convolutional layer, a way of searching for a subpattern over the whole of the image. We can chain multiple of these convolutional layers together, with the below example having two.

Next comes a pooling layer. This is a _downsampling_ technique, which effectively serves to reduce sample size and simplify later processes. For each value generated by our convolutional layers, it looks over the grid in _non_-overlapping segments and takes the maximum value of those outputs. It's not the feautres exact location then that matters, but its approximate or relative location. After pooling you will want to flatten the data back out, so that it can be put into dense layers as we did in MLP.

In [8]:
# input image dimensions, from our data
img_rows, img_cols = 28, 28
num_classes = 10

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 266s 4ms/step - loss: 0.2670 - acc: 0.9187 - val_loss: 0.0628 - val_acc: 0.9788
Epoch 2/10
60000/60000 [==============================] - 516s 9ms/step - loss: 0.0907 - acc: 0.9734 - val_loss: 0.0406 - val_acc: 0.9870
Epoch 3/10
60000/60000 [==============================] - 282s 5ms/step - loss: 0.0661 - acc: 0.9807 - val_loss: 0.0378 - val_acc: 0.9872
Epoch 4/10
60000/60000 [==============================] - 269s 4ms/step - loss: 0.0536 - acc: 0.9837 - val_loss: 0.0316 - val_acc: 0.9890
Epoch 5/10
60000/60000 [==============================] - 279s 5ms/step - loss: 0.0477 - acc: 0.9858 - val_loss: 0.0314 - val_acc: 0.9894
Epoch 6/10
60000/60000 [==============================] - 250s 4ms/step - loss: 0.0424 - acc: 0.9871 - val_loss: 0.0278 - val_acc: 0.9907
Epoch 7/10
60000/60000 [=========================

Now that is incredibly impressive accuracy. 99% is really exceptional, but it did take a long time to get there. Such are the costs of convolution.

There is one more classic construction of a neural network: Recurrent, which we'll give quick mention.

## Hierarchical Recurrrent Neural Networks

So far when we've talked about neural networks we've talked about them as feedforward: data flows in one direction until it reaches the end. Recurrent neural networks do not obey that directional logic, instead letting the data cycle through the network.

However, to do this we have to abandon the sequential model building we've done so far and things can get much more complicated. You have to use recurrent layers and often time distribution (which handles the extra dimension created through the LTSM layer, as a time dimension) to get these things running. You can find an example of a hierarchical recurrent network below (via the link [here](https://github.com/fchollet/keras/blob/master/examples/mnist_hierarchical_rnn.py)). When you get comfortable with networks as they exist in Keras for both convolution and MLP, start exploring recurrence. Note that this will take an even longer time than the previous ones should you choose to run it again.

In [9]:
# Training parameters.
batch_size = 64
num_classes = 10
epochs = 3

# Embedding dimensions.
row_hidden = 32
col_hidden = 32

# The data, shuffled and split between train and test sets.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshapes data to 4D for Hierarchical RNN.
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Converts class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

row, col, pixel = x_train.shape[1:]

# 4D input.
x = Input(shape=(row, col, pixel))

# Encodes a row of pixels using TimeDistributed Wrapper.
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)

# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 207s 3ms/step - loss: 1.0345 - acc: 0.6463 - val_loss: 0.5636 - val_acc: 0.8022
Epoch 2/3
60000/60000 [==============================] - 224s 4ms/step - loss: 0.4429 - acc: 0.8548 - val_loss: 0.3592 - val_acc: 0.8934
Epoch 3/3
60000/60000 [==============================] - 223s 4ms/step - loss: 0.2522 - acc: 0.9234 - val_loss: 0.2290 - val_acc: 0.9265
Test loss: 0.22904734394848347
Test accuracy: 0.9265


You should now be comfortable building some neural networks, but let's see if you can improve them!

# Drill: 99% MLP

We have the MLP above, which runs reasonably quickly. Copy that code down here and see if you can tune it to achieve 99% accuracy with a Multi-Layer Perceptron. Does it run faster than the recurrent or concolutional neural nets?